##### Malwina Wojewoda
### Task 1: Problem of linearly separable classes.

Dataset `earthquake.txt` corresponds to problem of prediction of seismic shocks (volcanic
eruptions and nuclear explosions) (variable popn) based on two variables: body (deep
wave magnitude) and surface (surface wave magnitude).

In [145]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from pypdf import PdfWriter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, mean_squared_error

In [146]:
df = pd.read_csv('data/earthquake.txt', sep=' ')

#### Make scatterplot for variables body and surface. Mark classes corresponding to observations.

In [147]:
fig = px.scatter(df, x='body', y='surface', color='popn', title='Scatterplot for variables body and surface')
fig.show()

#### Fit logistic model without regularization, print estimated coefficients, estimated probabilities and compute log-likelihood function.

In [148]:
X = df.drop('popn', axis=1)
y = np.array(df['popn'])

In [149]:
lr = LogisticRegression(penalty=None, random_state=17)
lr.fit(X, y)

LogisticRegression(penalty=None, random_state=17)

In [150]:
print(f"Coefficients: {lr.coef_[0]}")
print(f"Intercept: {lr.intercept_[0]}")
print(f"Probabilities: {lr.predict_proba(X)[:, 1]}")

Coefficients: [ 111.91608035 -117.54515973]
Intercept: -135.0476590763224
Probabilities: [3.76378816e-04 3.14508242e-08 2.71719066e-74 4.55919924e-28
 6.73136569e-96 2.35439841e-29 1.92311837e-68 4.02300724e-53
 1.32931825e-79 1.43811319e-44 2.33570214e-28 1.18751053e-42
 3.62956858e-04 2.39254786e-09 3.04157444e-47 4.03167927e-62
 4.03362280e-55 5.07501707e-46 1.54564287e-68 3.33030492e-75
 1.00000000e+00 9.99999961e-01 9.99269321e-01 1.00000000e+00
 1.00000000e+00 1.00000000e+00 9.99987751e-01 1.00000000e+00
 1.00000000e+00]


In [151]:
log_likelihood = -log_loss(y, lr.predict_proba(X)[:, 1])
print(f"Log-Likelihood: {log_likelihood}")

Log-Likelihood: -5.112896345885174e-05


#### Fit logistic model with L2 regularization, print estimated coefficients, estimated probabilities and compute log-likelihood function

In [152]:
lr = LogisticRegression(penalty='l2', random_state=17)
lr.fit(X, y)
print(f"Coefficients: {lr.coef_[0]}")
print(f"Intercept: {lr.intercept_[0]}")
print(f"Probabilities: {lr.predict_proba(X)[:, 1]}")
log_likelihood = -log_loss(y, lr.predict_proba(X)[:, 1])
print(f"Log-Likelihood: {log_likelihood}")

Coefficients: [ 2.16922885 -1.18171354]
Intercept: -7.469377343160523
Probabilities: [0.41484342 0.29381932 0.2268229  0.24081719 0.10988436 0.21714071
 0.09269171 0.08587458 0.08693786 0.1306053  0.22289096 0.17082695
 0.27057856 0.29712216 0.12950671 0.09811039 0.11986513 0.0832425
 0.07864525 0.10282201 0.62619915 0.5727785  0.51461215 0.54937943
 0.65820678 0.79354187 0.49879129 0.66401092 0.64941609]
Log-Likelihood: -0.29051987080152314


### Task 2: Simulation example
Generate data from logistic model.
Fit logistic model and calculate the estimators of the coefficients.
Repeat the experiment L = 100 times and compute the MSE.

In [153]:
def generate_data(n = 50):
    X = np.random.normal(0, 1, size=(n, 5))
    beta_0 = 0.5
    betas = np.array([1, 1, 1, 1, 1])
    y = [np.random.binomial(n=1, p=1/(1 + np.exp(-(beta_0 + np.dot(x, betas))))) for x in X]
    return X, y

In [154]:
def first_experiment(n = 50):
    X, y = generate_data(n)
    L = 100
    all_betas = np.array([0.5, 1, 1, 1, 1, 1])
    estimated_betas = np.zeros((L, len(all_betas)))
    for i in range(L):
        lr = LogisticRegression()
        lr.fit(X, y)
        estimated_betas[i] = np.insert(lr.coef_[0], 0, lr.intercept_[0])
    all_betas_array = np.tile(all_betas, (L, 1))
    return mean_squared_error(estimated_betas, all_betas_array)

In [155]:
mse = first_experiment()
print(f"MSE: {mse}")

MSE: 0.23742264118939616


Repeat the experiment for n = 50, 60, 70, 80, 90, 100, 200, 300 . . . , 1000 and make a plot showing how MSE depends on n.

In [156]:
n_values = [50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
mse_values = []
for n in n_values:
    mse = first_experiment(n)
    mse_values.append(mse)

In [157]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=n_values, y=mse_values, mode='lines+markers'))
fig.update_layout(title='Relationship between n (number of observations) and MSE <br>for model trained on 5 variables',
                  xaxis_title='n',
                  yaxis_title='MSE')
pio.write_image(fig, file=f'assets/task_2/SimulationResult5Variables.pdf', format='pdf')
fig.show()

Using the same datasets, train the model based only on 3 variables: x_{i,1}, x_{i,2}, x_{i,3} and draw the analogous curve showing how MSE for β = (β1, β2, β3) depends on n.

In [158]:
def third_experiment(n = 50):
    X, y = generate_data(n)
    L = 100
    all_betas = np.array([0.5, 1, 1, 1])
    estimated_betas = np.zeros((L, len(all_betas)))
    for i in range(L):
        lr = LogisticRegression()
        lr.fit(X[:, :3], y)
        estimated_betas[i] = np.insert(lr.coef_[0], 0, lr.intercept_[0])
    all_betas_array = np.tile(all_betas, (L, 1))
    return mean_squared_error(estimated_betas, all_betas_array)

In [159]:
n_values = [50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
mse_values = []
for n in n_values:
    mse = third_experiment(n)
    mse_values.append(mse)

In [160]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=n_values, y=mse_values, mode='lines+markers'))
fig.update_layout(title='Relationship between n (number of observations) and MSE <br>for model trained on 3 variables',
                  xaxis_title='n',
                  yaxis_title='MSE')
pio.write_image(fig, file=f'assets/task_2/SimulationResult3Variables.pdf', format='pdf')
fig.show()

In [161]:
merger = PdfWriter()
merger.append("assets/task_2/SimulationResult5Variables.pdf") 
merger.append("assets/task_2/SimulationResult3Variables.pdf")
merger.write("assets/task_2/SimulationResult.pdf")
merger.close()